In [1]:
import os
import openai
import pandas as pd
import numpy as np
import lmdb
import io
import json

client = openai.OpenAI()

In [2]:
def get_embedding(content, model='text-embedding-ada-002'):
    response = client.embeddings.create(input=content, model=model)
    vector = response.data[0].embedding
    return vector

In [3]:
def make_vector_db(df):
    with lmdb.open('.', map_size=10485760) as env:
        with env.begin(write=True) as txn:
            for id, [question, answer] in df[["question", "answer"]].iterrows():
                question = json.dumps(get_embedding(question))
            
                txn.put(f"EQ_{id}".encode(), question.encode())
                txn.put(f"A_{id}".encode(), answer.encode())
            
            txn.put(b"num_samples", str(len(df)).encode())
            
df = pd.read_excel("./../crawling/result.xlsx")
make_vector_db(df)

In [42]:
import lmdb

def get_all_key():
    with lmdb.open('./') as env:
        with env.begin() as txn:
            num = int(txn.get(b'num_samples').decode())
            return [f"EQ_{i}".encode() for i in range(num)]
        
get_all_key()


AttributeError: 'NoneType' object has no attribute 'decode'

In [45]:
env.close()

In [33]:
def get():
    env = lmdb.open('.', map_size=10485760)
    with env.begin() as txn:
        question = json.loads(txn.get(f"EQ_{141}".encode()).decode())
        answer = txn.get(f"A_{141}".encode()).decode()
        print(question, answer) 
        
get()

[-0.011834652163088322, -0.022106973454356194, 0.02334381826221943, -0.009653898887336254, -0.01939893327653408, 0.005888032261282206, -0.009419549256563187, 0.01692524366080761, -0.011619831435382366, -0.025635235011577606, -0.02067483775317669, 0.04002169519662857, -0.007271345239132643, -0.01081913709640503, 0.004042529501020908, 0.02288813889026642, 0.006493435241281986, -0.022172071039676666, 0.010721491649746895, -0.006307908333837986, -0.008762069046497345, -0.0020228922367095947, 0.0039123352617025375, 0.0024639249313622713, -0.01601388491690159, 0.006978408433496952, 0.028043828904628754, -0.006285124458372593, -0.0022507319226861, 0.02615601196885109, 0.006594335660338402, -0.005793641321361065, -0.025778450071811676, 0.006919821258634329, -0.005513723939657211, 0.01069545280188322, 0.005809915717691183, 0.013969836756587029, 0.03387652710080147, -0.03036128543317318, 0.004879027139395475, -0.0042345658876001835, 0.015167622826993465, 0.004172723740339279, -0.0036812406033277

In [11]:


df["question"].apply(lambda x: get_embedding(x))

0      [0.0046925595961511135, -0.03953813761472702, ...
1      [-0.006125828716903925, -0.024051029235124588,...
2      [0.00818714965134859, -0.01991540752351284, 0....
3      [0.024739135056734085, -0.02889719419181347, 0...
4      [0.003631087252870202, -0.02928745560348034, -...
                             ...                        
137    [0.013370271772146225, -0.023690948262810707, ...
138    [-0.017034780234098434, -0.03236345946788788, ...
139    [0.015900060534477234, -0.030010726302862167, ...
140    [-0.0008823047974146903, -0.026621835306286812...
141    [-0.011834652163088322, -0.022106973454356194,...
Name: question, Length: 142, dtype: object

In [9]:
df["question"].apply(lambda x : x[:3])

0      생산에
1      총수요
2      노동시
3      24번
4      교재와
      ... 
137    15번
138    박봄 
139    기회비
140    202
141    시장의
Name: question, Length: 142, dtype: object

In [ ]:
def similarity(v1, v2):  # return dot product of two vectors
    return np.dot(v1, v2)

In [12]:
len(get_embedding("hello"))

1536